In [ ]:
%matplotlib inline

Solving linear problems - advanced
==================================

::: {.seealso}
See
`sphx_glr_auto_examples_solvers_ex_linear_problems_basic.py`{.interpreted-text
role="ref"} for an introduction to solving linear problems. See
`sphx_glr_auto_examples_solvers_ex_quad_problems_basic.py`{.interpreted-text
role="ref"} for an introduction to solving quadratic problems. See
`sphx_glr_auto_examples_solvers_ex_quad_problems_advanced.py`{.interpreted-text
role="ref"} for an advanced example how to solve quadratic problems.
:::


This examples is a continuation of TODO reference and shows advanced
examples for how to solve linear problems like the
`moscot.problems.time.TemporalProblem`{.interpreted-text role="class"}
and the `moscot.problems.generic.SinkhornProblem`{.interpreted-text
role="class"}.


In [ ]:
from moscot.datasets import simulate_data
from moscot.problems.generic import SinkhornProblem

adata = simulate_data(n_distributions=2, key="day")
adata

Threshold
=========

The [threshold]{.title-ref} parameter defines the convergence criterion.
In the balanced setting the [threshold]{.title-ref} denotes the
deviation between prior and posterior marginals, while in the unbalanced
setting the [threshold]{.title-ref} corresponds to a Cauchy sequence
stopping criterion.

Initializers
============

Different Initializers can help to improve convergence. For the
full-rank case we can set the initializer to the trivial initializing
method denoted by [default]{.title-ref}. The [gaussian]{.title-ref}
(`thornton:22`{.interpreted-text role="cite"}) initializer computes
Gaussian approximations of two point clouds and leverages the
closed-form solution of Optimal Transport problems between Gaussians,
while the [sorting]{.title-ref} initializer
(`thornton:22`{.interpreted-text role="cite"}) solves a simplified
(sorting) Optimal Transport problem and uses its solution as
initializer. See `ott.core.initializers`{.interpreted-text role="mod"}
for details.

For low-rank problems different initializers are available, see
`ott.core.initializers_lr`{.interpreted-text role="mod"}. Initialization
can be [random]{.title-ref}, [rank2]{.title-ref}
(`scetbon:22b`{.interpreted-text role="cite"}), [k-means]{.title-ref},
or [generalized-k-means]{.title-ref}. For some initializers keyword
arguments can be provided as a dictionary, e.g.
[min\_iterations]{.title-ref} and [max\_iterations]{.title-ref} can be
provided for the k-means algorithm used by the [k-means]{.title-ref}
initializer.


In [ ]:
sp = SinkhornProblem(adata)
sp = sp.prepare(key="day")

ik = {"min_iterations": 5, "max_iterations": 200}
sp = sp.solve(epsilon=0, rank=3, initializer="k-means", initializer_kwargs=ik)

Number of iterations
====================

There are three types of iterations, which can be set.
[min\_iterations]{.title-ref} is the minimum number of iterations of the
algorithm. [max\_iterations]{.title-ref} is the maximum number of
iterations. If the convergence criterion is not met after completing
[max\_iterations]{.title-ref}, the model has not converged.
[inner\_iterations]{.title-ref} is the number of iterations after which
the model checks the convergence criterion. If
[max\_iterations]{.title-ref} is too low, the model won\'t converge:


In [ ]:
sp = sp.solve(epsilon=1e-3, inner_iterations=1, min_iterations=0, max_iterations=2)

Low rank hyperparameters
========================

The low-rank algorithm requires more hyperparameters, i.e.
[gamma]{.title-ref}, the a step size of the mirror descent algorithm and
[gamma\_rescale]{.title-ref}, a flag indicating whether to rescale
[gamma]{.title-ref}. When tuning [gamma]{.title-ref}, we recommend
trying orders of 10. If [gamma]{.title-ref} is too small or too large,
the algorithm might not converge


In [ ]:
sp = sp.solve(epsilon=0, rank=3, initializer="random", max_iterations=30, gamma=1000)

In [ ]:
sp = sp.solve(epsilon=0, rank=3, initializer="random", max_iterations=30, gamma=10)

Keyword arguments & Implementation details
==========================================

Whenever a [solve]{.title-ref} method of a linear problem is called, a
backend-specific linear solver is instantiated. Currently,
`ott`{.interpreted-text role="mod"} is supported, its corresponding
linear solvers are `ott.core.sinkhorn.Sinkhorn`{.interpreted-text
role="class"}, which is used whenever [rank = -1]{.title-ref}, and
`ott.core.sinkhorn_lr.LRSinkhorn`{.interpreted-text role="class"}, its
counterpart whenever [rank]{.title-ref} is a positive integer.
`moscot`{.interpreted-text role="mod"} wraps these classes in
`moscot.backends.ott.SinkhornSolver`{.interpreted-text role="class"}
which handles both full and low rank.
